In [ ]:
import hypermri
import hypermri.utils.utils_anatomical as ut_anat
import sys

import os
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import pandas as pd
import datetime

from scipy.optimize import curve_fit 
import matplotlib


from hypermri.utils.utils_sv_spectroscopy import Plot_Voxel_on_Anat
# Autoreload extension so that you dont have to reload the kernel every time something is changed in the hypermri or magritek folders
%load_ext autoreload
%autoreload 2

%matplotlib widget

def get_colors_from_cmap(cmap_name, N):
    cmap = plt.get_cmap(cmap_name)
    colors = cmap(np.linspace(0, 1, N))
    return colors


from matplotlib import rc
rc("font", **{"family": "serif", "serif": ["Computer Modern"]})
rc("text", usetex=True)
matplotlib.rcParams.update({"font.size": 11})
import hypermri


### Load data

In [ ]:
# define scan path to folder where measurements are
dirpath = ''
scans = hypermri.BrukerDir(dirpath)

In [ ]:
figuresize=6.9

In [ ]:
sv_press=[scans[n] for n in range(13,18,1)]
mvslaser_self_optimal = scans[11]
mvpress = scans[12]
mvslaser_bruker=scans[10]

# Plot spectra

In [ ]:
plt.close('all')
fig,ax=plt.subplots(2,2,figsize=(figuresize/1.5,figuresize/2),tight_layout=True)

offset=3.37
voxel_pos = ['Water','Oil','Ethanol','Outside']
colors=['C0','C1','C2','C3']
letters=['A','B','C','D']
ppm_slaser=hypermri.utils.utils_spectroscopy.get_freq_axis(mvslaser_self_optimal)
ppm_press=hypermri.utils.utils_spectroscopy.get_freq_axis(mvpress)

for voxel in range(4):
    nx,ny=voxel//2,voxel%2
    noise_std_slaser=np.std(np.abs(np.squeeze(mvslaser_self_optimal.complex_spec[0:100,voxel])))
    noise_std_press=np.std(np.abs(np.squeeze(mvpress.complex_spec[0:100,voxel])))
    
    color=colors[voxel]
    ax[nx,ny].plot(ppm_slaser-offset,np.abs(np.squeeze(mvslaser_self_optimal.complex_spec[:,voxel]))/noise_std_slaser,label='MV-sLASER',color='r')
    ax[nx,ny].plot(ppm_press-offset,np.abs(np.squeeze(mvpress.complex_spec[:,voxel]))/noise_std_press,color='k',linestyle='solid',label='MV-PRESS')
    ax[nx,ny].set_xlim([6,-1])

    ax[nx,ny].set_title(letters[voxel],loc='left')

    
ax[0,0].set_xticks([])
ax[0,1].set_xticks([])
[ax[1,n].set_xlabel(r'$\sigma $[ppm]') for n in range(2)]

ax[0,0].set_ylabel('SNR')
ax[1,0].set_ylabel('SNR')

ax[0,0].set_yticks([0,3000])
ax[0,1].set_yticks([0,1000])
ax[0,1].set_ylim([-40,1300])
ax[1,0].set_ylim([-40,650])
ax[1,1].set_ylim([-2,32])

ax[1,0].set_yticks([0,500])
ax[1,1].set_yticks([0,25])
ax[1,1].legend(fontsize=9)
fig.subplots_adjust(wspace=0,hspace=0)

In [ ]:
fig,ax=plt.subplots(2,1,figsize=(2.56,figuresize/2),tight_layout=True)


ppm_slaser=hypermri.utils.utils_spectroscopy.get_freq_axis(mvslaser_self_optimal)
ppm_press=hypermri.utils.utils_spectroscopy.get_freq_axis(mvpress)

for voxel in [0,1]:
    max_peak_slaser=np.max(np.abs(np.squeeze(mvslaser_self_optimal.complex_spec[:,voxel])))
    max_peak_press=np.max(np.abs(np.squeeze(mvpress.complex_spec[:,voxel])))
    
    ax[voxel].plot(ppm_slaser-offset,np.abs(np.squeeze(mvslaser_self_optimal.complex_spec[:,voxel]))/max_peak_slaser,label='MV-sLASER',color='r')
    ax[voxel].plot(ppm_press-offset,np.abs(np.squeeze(mvpress.complex_spec[:,voxel]))/max_peak_press,color='k',linestyle='solid',label='MV-PRESS')

    ax[voxel].set_ylabel(r'$I_{norm}$ [a.u.]')
    ax[voxel].set_yticks([0,1])
    
ax[0].set_xlim([7,2])
ax[0].set_ylim([-0.002,0.0385])

ax[1].set_xlim([3,-0.3])
ax[1].set_ylim([-0.01,0.211])
ax[1].set_xlabel(r'$\sigma $[ppm]')
ax[0].set_yticks([0,0.03])
ax[1].set_yticks([0,0.2])


# Plot voxel positions

In [ ]:
axial=scans[5]
coronal=scans[6]

In [ ]:
fig,ax=plt.subplots(2,2,tight_layout=True,figsize=(figuresize/2,figuresize/2))

Plot_Voxel_on_Anat(mvslaser_self_optimal,axial,ax[0,0],0)
Plot_Voxel_on_Anat(mvslaser_self_optimal,axial,ax[0,1],1)
Plot_Voxel_on_Anat(mvslaser_self_optimal,axial,ax[1,0],2)
Plot_Voxel_on_Anat(mvslaser_self_optimal,axial,ax[1,1],3)

ax[0,0].set_title('water')
ax[0,1].set_title('oil')
ax[1,0].set_title('ethanol')
ax[1,1].set_title('outside')


ax[0,0].axis('off')
ax[1,0].axis('off')
ax[0,1].axis('off')
ax[1,1].axis('off')

# Plot slice profile data

In [ ]:
# define scan path
dirpath = basepath + ''
scans = hypermri.BrukerDir(dirpath,verbose=True)
axial=scans[5]

In [ ]:
slaser_slic_prof=scans[11] # 5mm
press_slic_prof=scans[9] # 5mm

In [ ]:
plt.close('all')
fig,ax=plt.subplots(2,2,tight_layout=True,figsize=(figuresize/2,figuresize/2))

ax[0,0].set_title('PRESS')
ax[1,0].set_title('sLASER')
for n in range(0,2,1):
    ax[n,0].set_xlim([-5,5])
    ax[n,0].set_ylim([-5,5])
    ax[n,1].set_xlim([-5,5])
width_press = press_slic_prof.method['PVM_SliceThick']
width_slaser = slaser_slic_prof.method['PVM_SliceThick']


print(width_press,width_slaser)
fov=press_slic_prof.method['PVM_Fov'][0]
ax[0,0].imshow(press_slic_prof.seq2d,extent=press_slic_prof.get_extent('main'),cmap='magma')
ax[1,1].plot(np.linspace(-fov/2,fov/2,140),np.mean(press_slic_prof.seq2d[:,:,0],axis=1)/np.max(np.mean(press_slic_prof.seq2d[:,:,0],axis=1)),label='PRESS',color='k')
ax[0,1].plot(np.linspace(-fov/2,fov/2,140),np.mean(press_slic_prof.seq2d[:,:,0],axis=0)/np.max(np.mean(press_slic_prof.seq2d[:,:,0],axis=0)),label='PRESS',color='k')

rect = Rectangle((-width_press/2,-width_press/2),width_press,width_press,fc='None',ec='w')
ax[0,0].add_patch(rect)

ax[1,0].imshow(slaser_slic_prof.seq2d,extent=slaser_slic_prof.get_extent('main'),cmap='magma')
ax[1,1].plot(np.linspace(-fov/2,fov/2,140),np.mean(slaser_slic_prof.seq2d[:,:,0],axis=1)/np.max(np.mean(slaser_slic_prof.seq2d[:,:,0],axis=1)),label='sLASER',color='r')
ax[0,1].plot(np.linspace(-fov/2,fov/2,140),np.mean(slaser_slic_prof.seq2d[:,:,0],axis=0)/np.max(np.mean(slaser_slic_prof.seq2d[:,:,0],axis=0)),label='sLASER',color='r')

rect = Rectangle((-width_slaser/2,-width_slaser/2),width_slaser,width_slaser,fc='None',ec='w')
ax[1,0].add_patch(rect)

#ax[0,0].set_yticks([])
#ax[1,0].set_yticks([])
ax[1,1].set_yticks([0,1])
ax[0,1].set_yticks([0,1])

ax[1,0].set_xlabel('mm')
ax[1,1].set_xlabel('mm')
ax[0,0].set_ylabel('mm')
ax[1,0].set_ylabel('mm')


ax[0,1].set_ylabel('I [a.u.]')
ax[1,1].set_ylabel('I [a.u.]')

ax[0,1].legend(fontsize=8)
ax[1,1].legend(fontsize=8)

